# Hierarchical Modeling
What models fit the data?

# Setup

### Imports

In [1]:
# Calculation imports
import cmdstanpy
import numpy as np
import pandas as pd
import scipy
from scipy.stats import norm, binned_statistic
from sklearn.metrics import root_mean_squared_error
import sklearn.metrics as sk_metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from tqdm import tqdm

In [2]:
# Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns

### Parameters

In [3]:
config = {
    "bins": 16,
    "min_bin_percentile": 10,
    "eval_sample_size": 1000,
    "nonden_fit_cols": ["references", "year"],
    "cat_col": "stan_field_code",
}

In [4]:
palette = sns.color_palette()

In [5]:
def log_ppd(log_p):
    log_p_max = np.max(log_p)
    log_sum_exp = log_p_max + np.log(np.sum(np.exp(log_p - log_p_max)))
    return log_sum_exp - len(log_p)

### Load data

In [ ]:
df_all = pd.read_csv("../../analysis_data/all_data.csv")
df_all

In [ ]:
# Basic nan handling across all obs
df_all["log_cpy"] = np.log10(df_all["citations_per_year"])
df_all["log_cpy"] = df_all["log_cpy"].replace(-np.inf, np.nan)

In [ ]:
# Make the field category more useful
df_all["fields_of_study_0"] = df_all["fields_of_study_0"].astype("category")
df_all["stan_field_code"] = df_all["fields_of_study_0"].cat.codes + 1
field_names = df_all["fields_of_study_0"].cat.categories
field_names

In [ ]:
vectorizer_names = sorted(df_all["vectorizer"].unique())
vectorizer_names

# Visual Overview

### Brief aside on logscale plotting

In [ ]:
fig = plt.figure()
ax = plt.gca()

x = np.logspace(-3, 3, 100)
log10 = np.log10(x)
log10p = np.log10(1 + x)
log10p2 = np.log10(1 + x) - 1

ax.plot(
    x,
    log10,
    label="log10",
)
ax.plot(
    x,
    log10p,
    label="log10p",
)
ax.plot(
    x,
    log10p2,
    label="log10p - 1",
)

ax.legend()

ax.set_xscale("log")

### Overview Normalized 2D Histogram

In [11]:
def get_bins(x, n_bins=config["bins"]):
    # Get density bins
    bins = np.linspace(
        np.nanpercentile(x, config["min_bin_percentile"]),
        np.nanpercentile(x, 100 - config["min_bin_percentile"]),
        n_bins + 1,
    )
    return bins


density_bins = df_all.groupby("vectorizer")["density"].apply(get_bins).to_dict()
cpy_bins = get_bins(
    df_all["log_cpy"], n_bins=17
)  # The n_bins=17 is to ensure we don't accidentally flip axes

In [12]:
def custom_histplot(data, x, y, normed=True, *args, **kwargs):

    # Get the appropriate facets
    vectorizer = data["vectorizer"].unique()[0]
    field = data["fields_of_study_0"].unique()[0]

    density_bins = get_bins(data[x])

    if normed:
        hist2d, _, _ = np.histogram2d(data[x], data[y], bins=(density_bins, cpy_bins))
        density_hist, _ = np.histogram(data[x], bins=density_bins)
        hist2d_normed = hist2d / density_hist.reshape(1, -1).T

        # Plot the data
        plt.pcolormesh(
            density_bins,
            cpy_bins,
            hist2d_normed.T,
            shading="auto",
            cmap="viridis",
        )
    else:
        sns.histplot(data, x=x, y=y, bins=(density_bins, cpy_bins), *args, **kwargs)

In [ ]:
fg = sns.FacetGrid(data=df_all, row="vectorizer", col="fields_of_study_0", sharex=False)
fg.map_dataframe(custom_histplot, x="density", y="log_cpy", normed=True)
fg.set_titles("{row_name} | {col_name}")

### Focused Normalized Histogram

In [14]:
df = df_all.query("vectorizer in ['SciBERT', 'Word2Vec']")

In [ ]:
fg = sns.FacetGrid(data=df, row="vectorizer", col="fields_of_study_0", sharex=False)
fg.map_dataframe(custom_histplot, x="density", y="log_cpy", normed=True)
fg.set_titles("{row_name} | {col_name}")

# Statistical Modeling

Note to future self: cmdstanpy and cmdstan advertise conda as *the* way to install.
Don't listen to them.
I couldn't get it to compile when I used conda for anything cmdstanpy related.
I downloaded the repo and compiled manually, and used pip for cmdstanpy.

## Train-Test Split

In [ ]:
# Pivot to wide format so we can use the same train_test division
# and so we can use multiple densities at once
df_vectorizers = df_all.pivot_table(
    index="identifier", columns="vectorizer", values="density"
)
# For the nonden cols we can take the first because the other values are duplicates
nonden_cols = [
    "log_cpy",
    config["cat_col"],
] + config["nonden_fit_cols"]
df_others = df_all.groupby("identifier")[nonden_cols].first()
df_eval = pd.concat([df_others, df_vectorizers], axis="columns")
df_eval

In [17]:
# Drop all columns where log_cpy is na
df_eval = df_eval.dropna(subset="log_cpy")

In [ ]:
# Apply scaling (normalization by mean and sigma) to help with modelling
scaled_cols = (
    [
        "log_cpy",
    ]
    + config["nonden_fit_cols"]
    + vectorizer_names
)
df_eval.loc[:, scaled_cols] = df_eval[scaled_cols].apply(scale)
df_eval

In [19]:
# Downsample
if config["eval_sample_size"] is not None:
    df_eval = df_eval.sample(config["eval_sample_size"])

In [20]:
# Train-test split
df_train, df_test = train_test_split(df_eval)

In [21]:
# dictionary to store results in
results_dict = {}

## Base model
The base model is just a normal distribution.

In [22]:
stan_model = "base"

In [23]:
# Dict for storing output
output = {}

In [24]:
# Drop na
df_train_i = df_train.dropna(subset=["log_cpy"])
df_test_i = df_train.dropna(subset=["log_cpy"])

In [25]:
# Format data
data = {
    "N": len(df_train_i),
    "y": df_train_i["log_cpy"].values,
    "N_test": len(df_test_i),
    "y_test": df_test_i["log_cpy"].values,
}

In [26]:
# Compile model
model = cmdstanpy.CmdStanModel(stan_file=f"../stan_models/{stan_model}.stan")

In [ ]:
# Fit model
fit = model.sample(
    data=data,
)

In [28]:
# Parse output
draws = fit.draws_pd()

In [29]:
# Store parameters
output["alpha"] = draws["alpha"].median()
output["sigma"] = draws["sigma"].median()

In [30]:
# Calculate log posterior predictive density
output["log_ppd"] = log_ppd(draws["log_p"])

In [31]:
results_dict[stan_model] = pd.Series(output)

## Linear regression on density
$(c \sim \rho_t)$ vs $(c \sim \rho_v)$

In [32]:
stan_model = "reg"

### Fit

In [ ]:
# Dictionary to hold modeling output

# Loop through vectorizers
outputs_for_this_model = {}
for i, vectorizer_i in enumerate(vectorizer_names):
    output = {}

    # Drop na
    df_train_i = df_train.dropna(subset=[vectorizer_i, "log_cpy"])
    df_test_i = df_train.dropna(subset=[vectorizer_i, "log_cpy"])

    # Format data
    data = {
        "N": len(df_train_i),
        "x": df_train_i[vectorizer_i].values,
        "y": df_train_i["log_cpy"].values,
        "N_test": len(df_test_i),
        "x_test": df_test_i[vectorizer_i].values,
        "y_test": df_test_i["log_cpy"].values,
    }

    # Compile model
    model = cmdstanpy.CmdStanModel(stan_file=f"../stan_models/{stan_model}.stan")

    # Fit model
    fit = model.sample(
        data=data,
    )

    # Parse output
    draws = fit.draws_pd()

    # Store parameters
    output["alpha"] = draws["alpha"].median()
    output[f"beta[{vectorizer_i}]"] = draws["beta"].median()
    output["sigma"] = draws["sigma"].median()

    # Calculate log posterior predictive density
    output["log_ppd"] = log_ppd(draws["log_p"])

    key = f"{stan_model}_{vectorizer_i}"
    outputs_for_this_model[key] = output
    results_dict[key] = output

### Evaluate

In [34]:
# Format for showing here
output = pd.DataFrame(outputs_for_this_model).T
output["vectorizer"] = vectorizer_names

In [35]:
def medianplot(data, x, y, bins, ax):

    x = data[x]
    y = data[y]

    centers = (bins[1:] + bins[:-1]) / 2

    # Calculate running median
    median, bin_edges, bin_number = binned_statistic(
        x, y, statistic=np.nanmedian, bins=bins
    )
    ax.plot(
        centers,
        median,
        color="k",
    )

    # Calculate running percentiles
    low, bin_edges, bin_number = binned_statistic(
        x,
        y,
        statistic=lambda v: np.nanpercentile(v, 16),
        bins=bins,
    )
    high, bin_edges, bin_number = binned_statistic(
        x,
        y,
        statistic=lambda v: np.nanpercentile(v, 84),
        bins=bins,
    )
    ax.fill_between(
        centers,
        low,
        high,
        color="k",
        alpha=0.1,
    )

    ax.set_xlim(bins[0], bins[-1])

In [36]:
left_edge = np.nanpercentile(
    df_eval[vectorizer_names].values.flatten(), config["min_bin_percentile"]
)
right_edge = -left_edge
bins = np.linspace(left_edge, right_edge, config["bins"])

In [ ]:
for i, row in output.iterrows():
    fig = plt.figure()
    ax = plt.gca()

    df_plot = df_eval.dropna(subset=[row["vectorizer"], "log_cpy"])

    medianplot(
        data=df_plot,
        x=row["vectorizer"],
        y="log_cpy",
        bins=bins,
        ax=ax,
    )

    # Plot the regression
    xs = bins
    ys = row["alpha"] + row[f"beta[{row["vectorizer"]}]"] * xs
    ax.plot(
        xs,
        ys,
        color=palette[0],
    )
    ax.fill_between(
        xs,
        ys - row["sigma"],
        ys + row["sigma"],
        color=palette[0],
        alpha=0.4,
    )

    ax.set_xlabel(f"density [{row["vectorizer"]}]")
    ax.set_ylabel("log_cpy")

    

## Multivariate linear regression w/o density
The next model is a multivate linear regression with no density

In [38]:
stan_model = "multireg"
fit_cols = config["nonden_fit_cols"]
cols = [
    "log_cpy",
] + fit_cols

In [39]:
# Dict for storing output
output = {}

In [40]:
# Drop na
df_train_i = df_train[cols].dropna()
df_test_i = df_train[cols].dropna()

In [41]:
# Format data
data = {
    "N": len(df_train_i),
    "K": len(fit_cols),
    "x": df_train_i[fit_cols].values,
    "y": df_train_i["log_cpy"].values,
    "N_test": len(df_test_i),
    "x_test": df_test_i[fit_cols].values,
    "y_test": df_test_i["log_cpy"].values,
}

In [42]:
# Compile model
model = cmdstanpy.CmdStanModel(stan_file=f"../stan_models/{stan_model}.stan")

In [ ]:
# Fit model
fit = model.sample(
    data=data,
)

In [44]:
# Parse output
draws = fit.draws_pd()

In [45]:
# Store parameters
output["alpha"] = draws["alpha"].median()
output["sigma"] = draws["sigma"].median()
beta_cols = [col for col in draws.columns if col[:4] == "beta"]
betas = draws[beta_cols].median(axis="rows")
for i, fit_col in enumerate(fit_cols):
    output[f"beta[{fit_col}]"] = betas.iloc[i]

In [46]:
# Calculate log posterior predictive density
output["log_ppd"] = log_ppd(draws["log_p"])

In [47]:
results_dict[stan_model] = pd.Series(output)

## Multivariate linear regression
Now with density.

In [48]:
stan_model = "multireg"

In [ ]:
# Dictionary to hold modeling output

# Loop through vectorizers
outputs_for_this_model = {}
for i, vectorizer_i in enumerate(vectorizer_names):
    output = {}

    # Drop na
    fit_cols = [
        vectorizer_i,
    ] + config["nonden_fit_cols"]
    cols = [
        "log_cpy",
    ] + fit_cols
    df_train_i = df_train.dropna(subset=cols)
    df_test_i = df_train.dropna(subset=cols)

    # Format data
    data = {
        "N": len(df_train_i),
        "K": len(fit_cols),
        "x": df_train_i[fit_cols].values,
        "y": df_train_i["log_cpy"].values,
        "N_test": len(df_test_i),
        "x_test": df_test_i[fit_cols].values,
        "y_test": df_test_i["log_cpy"].values,
    }

    # Compile model
    model = cmdstanpy.CmdStanModel(stan_file=f"../stan_models/{stan_model}.stan")

    # Fit model
    fit = model.sample(
        data=data,
    )

    # Parse output
    draws = fit.draws_pd()

    # Store parameters
    output["alpha"] = draws["alpha"].median()
    output["sigma"] = draws["sigma"].median()
    beta_cols = [col for col in draws.columns if col[:4] == "beta"]
    betas = draws[beta_cols].median(axis="rows")
    for i, fit_col in enumerate(fit_cols):
        output[f"beta[{fit_col}]"] = betas.iloc[i]

    # Calculate log posterior predictive density
    output["log_ppd"] = log_ppd(draws["log_p"])

    key = f"{stan_model}_{vectorizer_i}"
    outputs_for_this_model[key] = output
    results_dict[key] = output

## Multivariate linear regression w/ all fields
The next model is a multivate linear regression using every variable we have

In [50]:
stan_model = "multireg"
fit_cols = config["nonden_fit_cols"] + vectorizer_names
cols = [
    "log_cpy",
] + fit_cols

In [51]:
# Dict for storing output
output = {}

In [52]:
# Drop na
df_train_i = df_train[cols].dropna()
df_test_i = df_train[cols].dropna()

In [53]:
# Format data
data = {
    "N": len(df_train_i),
    "K": len(fit_cols),
    "x": df_train_i[fit_cols].values,
    "y": df_train_i["log_cpy"].values,
    "N_test": len(df_test_i),
    "x_test": df_test_i[fit_cols].values,
    "y_test": df_test_i["log_cpy"].values,
}

In [54]:
# Compile model
model = cmdstanpy.CmdStanModel(stan_file=f"../stan_models/{stan_model}.stan")

In [ ]:
# Fit model
fit = model.sample(
    data=data,
)

In [56]:
# Parse output
draws = fit.draws_pd()

In [57]:
# Store parameters
output["alpha"] = draws["alpha"].median()
output["sigma"] = draws["sigma"].median()
beta_cols = [col for col in draws.columns if col[:4] == "beta"]
betas = draws[beta_cols].median(axis="rows")
for i, fit_col in enumerate(fit_cols):
    output[f"beta[{fit_col}]"] = betas.iloc[i]

In [58]:
# Calculate log posterior predictive density
output["log_ppd"] = log_ppd(draws["log_p"])

In [59]:
results_dict[f"full_{stan_model}"] = pd.Series(output)

## Hierarchical Linear Regression on density

In [60]:
stan_model = "hreg"

### Fit

In [ ]:
# Dictionary to hold modeling output

# Loop through vectorizers
outputs_for_this_model = {}
for i, vectorizer_i in enumerate(vectorizer_names):
    output = {}

    # Drop na
    df_train_i = df_train.dropna(subset=[vectorizer_i, "log_cpy"])
    df_test_i = df_train.dropna(subset=[vectorizer_i, "log_cpy"])

    # Format data
    data = {
        "D": 1,
        "N": len(df_train_i),
        "L": df_train_i[config["cat_col"]].max(),
        "y": df_train_i["log_cpy"].values,
        "ll": df_train_i[config["cat_col"]].values,
        "x": df_train_i[
            [
                vectorizer_i,
            ]
        ].values,
    }

    # Compile model
    model = cmdstanpy.CmdStanModel(stan_file=f"../stan_models/{stan_model}.stan")

    # Fit model
    fit = model.sample(
        data=data,
    )

    # Parse output
    stan_vars = fit.stan_variables()

    # Store parameters
    output["alpha"] = np.median(stan_vars["alpha"])
    output["sigma"] = np.median(stan_vars["sigma"])

    # Store per-field betas
    betas = np.nanmedian(stan_vars["beta"], axis=0)
    for i, betas_i in enumerate(betas):
        field_name_i = field_names[i]
        beta_i = betas_i[0]
        output[f"beta[{vectorizer_i},{field_name_i}]"] = beta_i

    # Calculate log posterior predictive density
    # output["log_ppd"] = log_ppd(draws["log_p"])

    key = f"{stan_model}_{vectorizer_i}"
    outputs_for_this_model[key] = output
    results_dict[key] = output

### Evaluate

In [62]:
# Format for showing here
output = pd.DataFrame(outputs_for_this_model).T
output["vectorizer"] = vectorizer_names

In [ ]:
fig = plt.figure(figsize=(6.4 * len(field_names), 4.8 * len(vectorizer_names)))
mosaic = [[f"{vectorizer_i}_{field_j}" for field_j in field_names] for vectorizer_i in vectorizer_names]
ax_dict = fig.subplot_mosaic(mosaic=mosaic)
for i, (model_i, row) in enumerate(output.iterrows()):
    vectorizer_i = row["vectorizer"]
    for j, field_j in enumerate(field_names):

        ax_key = f"{vectorizer_i}_{field_j}" 
        ax = ax_dict[ax_key]

        df_plot = df_eval.query(f"stan_field_code == {j+1}")

        medianplot(
            data=df_plot,
            x=row["vectorizer"],
            y="log_cpy",
            bins=bins,
            ax=ax,
        )

        # Plot the regression
        xs = bins
        ys = row["alpha"] + row[f"beta[{row["vectorizer"]},{field_j}]"] * xs
        ax.plot(
            xs,
            ys,
            color=palette[0],
        )
        ax.fill_between(
            xs,
            ys - row["sigma"],
            ys + row["sigma"],
            color=palette[0],
            alpha=0.4,
        )

        ax.set_xlabel(f"density [{row["vectorizer"]},{field_j}]")
        ax.set_ylabel("log_cpy")


        

# Summarize

In [ ]:
results = pd.DataFrame(results_dict).T
results

## Basic metrics

### Log Posterior Predictive Density

In [ ]:
fig = plt.figure()
ax = plt.gca()

x = np.arange(len(results))
ax.scatter(
    x,
    results["log_ppd"],
)

ax.set_xticks(x)
ax.set_xticklabels(results.index, rotation=45)

### RMSE

In [ ]:
beta_cols = [col for col in results.columns if col[:4] == "beta"]
betas = results[beta_cols].fillna(0.0)
x_cols = [col[5:-1] for col in beta_cols]
x_test = df_test[x_cols].fillna(0.0)

In [ ]:
# Calculate y_pred
y_pred = np.matmul(x_test.values, betas.T)
y_pred += results["alpha"].values.reshape(1, -1)
y_pred.index = df_test.index
y_pred

In [ ]:
# Calcualte rmse
results["rmse"] = y_pred.apply(lambda x: root_mean_squared_error(df_test["log_cpy"], x))

In [ ]:
fig = plt.figure()
ax = plt.gca()

x = np.arange(len(results))
ax.scatter(
    x,
    results["rmse"],
)

ax.set_xticks(x)
ax.set_xticklabels(results.index, rotation=45, ha="right")
ax.set_ylabel("rmse")

ax.set_ylim(0, ax.get_ylim()[1])

### Maximum slopes

In [ ]:
max_betas = results[beta_cols].max(axis="columns")

In [ ]:
fig = plt.figure()
ax = plt.gca()

x = np.arange(len(results))
ax.scatter(
    x,
    max_betas,
)

ax.set_xticks(x)
ax.set_xticklabels(results.index, rotation=45, ha="right")
ax.set_ylabel(r"max($\beta_i$)")

ax.set_ylim(0, ax.get_ylim()[1])

In [ ]:
# max slopes correlation with rmse
ax = sns.scatterplot(
    results,
    x=max_betas,
    y=1 - results["rmse"],
)
ax.set_xlim(0, ax.get_xlim()[1])
ax.set_ylim(0, ax.get_ylim()[1])

ax.set_xlabel(r"$\max(\beta_i)$")
ax.set_ylabel(r"1 - rmse")

### RMSE vs distance from center

In [ ]:
# Inspect rmse as a function of distance from the center
for model_i in results.index:

    row = results.loc[model_i]

    # Only do models that have a density dependence
    try:
        vectorizer_i = model_i.split("_")[-1]
        assert vectorizer_i in vectorizer_names
    except AssertionError:
        continue

    # Bin and labels setup
    bins = np.linspace(-3, 3, config["bins"])
    centers = 0.5 * (bins[:-1] + bins[1:])
    den_key = f"rho_{vectorizer_i}"
    cut_key = f"{den_key}_bin"
    model_key = f"y_pred_{model_i}"

    # Format data into a new df and group it
    df_test_to_group = pd.DataFrame()
    df_test_to_group[den_key] = df_test[vectorizer_i]
    df_test_to_group[model_key] = y_pred[model_i]
    df_test_to_group[cut_key] = pd.cut(df_test_to_group[den_key], bins)
    df_test_to_group["y_test"] = df_test["log_cpy"]
    df_grouped = df_test_to_group.groupby(cut_key)

    # Get mean predicted y per bin
    def get_mean_y_pred(df):
        if len(df) == 0:
            return np.nan
        return df[model_key].mean()

    mean_y_pred = df_grouped.apply(get_mean_y_pred)

    # Get rmse
    def get_rmse_of_df(df):
        if len(df) == 0:
            return np.nan
        return root_mean_squared_error(df["y_test"], df[model_key])

    rmse_per_bin = df_grouped.apply(get_rmse_of_df)

    # Also get rmse if y_pred was just the mean
    def get_rmse_from_mean(df):
        if len(df) == 0:
            return np.nan
        y_pred = np.zeros(len(df))  # Since we normalized to 0
        return root_mean_squared_error(df["y_test"], y_pred)

    rmse_from_mean = df_grouped.apply(get_rmse_from_mean)

    # And rmse expected if it was a straight improvement on the mean that scales with
    # slope
    min_rmse = rmse_from_mean - np.abs(centers * row[f"beta[{vectorizer_i}]"])

    # Plot
    fig, ax_dict = plt.subplot_mosaic(
        [
            [
                "hist",
            ],
            [
                "reg",
            ],
            [
                "reg",
            ],
            [
                "rmse",
            ],
            [
                "rmse",
            ],
        ],
        figsize=(8, 12),
    )

    # Hist plot
    ax = ax_dict["hist"]
    sns.histplot(
        x=df_test_to_group[den_key],
        ax=ax,
        bins=bins,
    )
    ax.set_xlim(bins[0], bins[-1])
    ax.set_title(model_i)
    ax.set_xlabel(None)

    # Regression plot
    ax = ax_dict["reg"]

    # Plot the median
    medianplot(
        data=df_test_to_group,
        x=den_key,
        y="y_test",
        bins=bins,
        ax=ax,
    )

    # Plot the regression
    xs = bins
    ys = row["alpha"] + row[f"beta[{vectorizer_i}]"] * xs
    ax.plot(
        xs,
        ys,
        color=palette[0],
    )
    ax.fill_between(
        xs,
        ys - row["sigma"],
        ys + row["sigma"],
        color=palette[0],
        alpha=0.25,
    )

    # Plot what y_pred actually is
    ax = sns.scatterplot(
        x=centers,
        y=mean_y_pred,
        ax=ax,
    )

    ax.set_xlabel(None)
    ax.set_ylabel("log_cpy")

    # RMSE plot
    ax = ax_dict["rmse"]
    ax = sns.scatterplot(
        x=centers,
        y=rmse_per_bin,
        ax=ax,
        label="rmse",
    )
    ax = sns.scatterplot(x=centers, y=rmse_from_mean, ax=ax, label="rmse(y_pred=0)")
    ax = sns.scatterplot(x=centers, y=min_rmse, ax=ax, label="min rmse")
    ax.legend()
    # Center line
    ax.axvline(
        0,
        c="0.2",
        linestyle="--",
    )
    # Global rmse line
    ax.axhline(
        results.loc[model_i, "rmse"],
        c="0.2",
        linestyle="--",
    )

    ax.set_ylabel("rmse")
    ax.set_xlabel(den_key)

    ax.set_xlim(bins[0], bins[-1])
    ax.set_ylim(0, ax.get_ylim()[1])
    fig

The issue is that RMSE is capturing two behaviors: distance from the mean and also width of the distribution.
The width of the distribution largely dominates over the distance from the mean.
So instead we may want a metric that is an evaluation of the distance median.
This might relate to Nathaniel's choice to bin.

## Many metrics

In [ ]:
metrics = {
    r"$R^2$": sk_metrics.r2_score,
    "1-MAE": lambda y_true, y_pred: 1 - sk_metrics.mean_absolute_error(y_true, y_pred),
    "1-RMSE": lambda y_true, y_pred: 1
    - sk_metrics.root_mean_squared_error(y_true, y_pred),
    "1-MAPE": lambda y_true, y_pred: 1
    - sk_metrics.mean_absolute_percentage_error(y_true, y_pred),
    "1-MedAE": lambda y_true, y_pred: 1
    - sk_metrics.median_absolute_error(y_true, y_pred),
    "1-MPinL": lambda y_true, y_pred: 1 - sk_metrics.mean_pinball_loss(y_true, y_pred),
    r"$D^2_{pin}$": sk_metrics.d2_pinball_score,
    r"$D^2_{abs}$": sk_metrics.d2_absolute_error_score,
}

In [ ]:
for metric_label, metric_fn in metrics.items():

    metric = y_pred.apply(lambda x: metric_fn(df_test["log_cpy"], x))

    # Plot
    fig = plt.figure()
    ax = plt.gca()

    x = np.arange(len(results))
    ax.scatter(
        x,
        metric,
    )

    ax.set_xticks(x)
    ax.set_xticklabels(results.index, rotation=45, ha="right")
    ax.set_ylabel(metric_label)